In [2]:
from tensorflow.python.keras.utils.data_utils import get_file
import numpy as np
import os
import cv2
import tensorflow as tf
import time

In [3]:
# Set seed to get reproducable results

np.random.seed(123)

In [10]:
class Detector:
    def __init__(self):
        pass
    
    def readClasses(self, classesFilePath):
        with open(classesFilePath, 'r') as f:
            self.classesList = f.read().splitlines()
            
            # Colours List
            self.colorList = np.random.uniform(low=0, high=255, size=(len(self.classesList), 3))
            
            print(len(self.classesList), len(self.colorList))
            
    def downloadModel(self, modelURL):
        fileName = os.path.basename(modelURL)
        self.modelName = fileName[:fileName.index('.')]
        
        # Download and store it from the model zoo
        self.cacheDir = './pretrained_models'
        os.makedirs(self.cacheDir, exist_ok = True)
        get_file(fname=fileName, 
                origin=modelURL,
                cache_dir=self.cacheDir,
                cache_subdir='checkpoints',
                extract=True)
        
    def setModel(self, modelName):
        self.cacheDir = './pretrained_models'
        self.modelName = modelName
        
    def loadModel(self):
        print("Loading Model " + self.modelName)
        tf.keras.backend.clear_session() # Release memory from older models
        self.model = tf.saved_model.load(os.path.join(self.cacheDir, 'checkpoints', self.modelName, "saved_model"))

        print("Model " + self.modelName + " loaded successfully...")

    def createBoundingBox(self, image, threshold = 0.5):
        inputTensor = cv2.cvtColor(image.copy(), cv2.COLOR_RGB2BGR)
        inputTensor = tf.convert_to_tensor(inputTensor, dtype=tf.uint8) # Investigate tf.tensor objects
        inputTensor = inputTensor[tf.newaxis, ...] # What is happening here?

        detections = self.model(inputTensor) # Get predictions

        bboxs = detections['detection_boxes'][0].numpy()
        classIndexes = detections['detection_classes'][0].numpy().astype(np.int32)
        classScores = detections['detection_scores'][0].numpy()

        imH, imW, imC = image.shape
        bboxIdx = tf.image.non_max_suppression(bboxs, classScores, max_output_size=50, iou_threshold=0.5, score_threshold=0.5)

        print(bboxIdx)
        
        
        if len(bboxIdx) != 0:
            for i in bboxIdx:
                bbox = tuple(bboxs[i].tolist())
                classConfidence = round(100*classScores[i])
                classIndex = classIndexes[i]

                classLabelText = self.classesList[classIndex]
                classColor = self.colorList[classIndex]

                displayText = f'{classLabelText}: {classConfidence}%'

                ymin, xmin, ymax, xmax = bbox
                xmin, xmax, ymin, ymax = (xmin * imW, xmax*imW, ymin*imH, ymax*imH)
                xmin, xmax, ymin, ymax =  int(xmin), int(xmax), int(ymin), int(ymax)
                
                cv2.rectangle(image, (xmin, ymin), (xmax, ymax), color = classColor, thickness=1)
                cv2.putText(image, displayText, (xmin, ymin-10), cv2.FONT_HERSHEY_PLAIN, 1, classColor, thickness=2)
                
        return image
                
            
            
    def predictImage(self, imagePath, threshold):
        image = cv2.imread(imagePath)
        print(image)
        bboxImage = self.createBoundingBox(image, threshold)

        cv2.imwrite(self.modelName + ".jpg", bboxImage)
        cv2.imshow("Result", image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
    def predictVideo(self, videoPath, threshold = 0.5):
        cap = cv2.VideoCapture(videoPath)
        
        if(cap.isOpened() == False):
            print("Error opening file...")
            
        (success, image) = cap.read()

        startTime = 0

        while success:
            currentTime = time.time()

            fps = 1/(currentTime-startTime)
            startTime = currentTime
            bboxImage = self.createBoundingBox(image, threshold)
            cv2.putText(bboxImage, 'FPS: '+ str(int(fps)), (20, 70), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2)
            cv2.imshow('Result', bboxImage)

            key = cv2.waitKey(1) & 0xFF
            if key ==ord('q'):
                break

            (success, image) = cap.read()
                
        cv2.destroyAllWindows()
            

In [4]:
classFile = 'coco.names'
detector = Detector()
detector.readClasses(classFile)

modelURL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz'

# modelURL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d4_coco17_tpu-32.tar.gz'

92 92


In [ ]:
detector.downloadModel(modelURL)
detector.loadModel()

imagePath = 'test/3.jpg'
# videoPath = 'test/video-2.mp4' #0 for webcam
# videoPath = 0
threshold = 0.5
detector.predictImage(imagePath, threshold)
# detector.predictVideo(videoPath, threshold)

Loading Model ssd_mobilenet_v2_320x320_coco17_tpu-8
Model ssd_mobilenet_v2_320x320_coco17_tpu-8 loaded successfully...
[[[ 37  37  53]
  [ 38  38  54]
  [ 39  39  55]
  ...
  [ 22  27  42]
  [ 23  28  43]
  [ 23  28  43]]

 [[ 37  37  53]
  [ 38  38  54]
  [ 39  39  55]
  ...
  [ 22  27  42]
  [ 22  27  42]
  [ 23  28  43]]

 [[ 37  36  52]
  [ 38  37  53]
  [ 39  38  54]
  ...
  [ 22  27  42]
  [ 22  27  42]
  [ 22  27  42]]

 ...

 [[134 160 190]
  [133 159 189]
  [132 158 188]
  ...
  [104 152 186]
  [103 151 185]
  [105 153 187]]

 [[135 161 191]
  [134 160 190]
  [133 159 189]
  ...
  [102 150 184]
  [100 148 182]
  [103 151 185]]

 [[136 162 192]
  [134 160 190]
  [134 160 190]
  ...
  [ 99 147 181]
  [ 96 144 178]
  [100 148 182]]]
tf.Tensor([0], shape=(1,), dtype=int32)
[[[ 37  37  53]
  [ 38  38  54]
  [ 39  39  55]
  ...
  [ 22  27  42]
  [ 23  28  43]
  [ 23  28  43]]

 [[ 37  37  53]
  [ 38  38  54]
  [ 39  39  55]
  ...
  [ 22  27  42]
  [ 22  27  42]
  [ 23  28  43]]

 [[

In [11]:
classFile = 'sign-language.names'
detector = Detector()
detector.readClasses(classFile)

modelName = 'my_model'

# modelURL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d4_coco17_tpu-32.tar.gz'

7 7


In [19]:
detector.setModel(modelName)
detector.loadModel()

imagePath = r"C:\Users\sttas\OneDrive\Documents\sign-language\training_demo\images\train\thumbsup.a97238da-04af-11ed-a1fb-9078414817bd.jpg"
# videoPath = 'test/video-2.mp4' #0 for webcam
# videoPath = 0
threshold = 0.2
detector.predictImage(imagePath, threshold)
# detector.predictVideo(videoPath, threshold)

Loading Model my_model
Model my_model loaded successfully...
[[[239 243 248]
  [240 243 247]
  [244 244 244]
  ...
  [106 101 102]
  [106 101 103]
  [106 101 103]]

 [[239 243 248]
  [240 243 247]
  [244 244 244]
  ...
  [106 101 102]
  [106 101 103]
  [106 101 103]]

 [[240 243 248]
  [242 243 247]
  [244 244 244]
  ...
  [105 100 101]
  [105 100 102]
  [105 100 102]]

 ...

 [[131 130 132]
  [132 131 133]
  [133 132 134]
  ...
  [195 165 146]
  [198 166 147]
  [206 175 154]]

 [[130 129 133]
  [131 130 134]
  [132 131 133]
  ...
  [197 164 149]
  [202 169 154]
  [213 178 164]]

 [[129 128 132]
  [131 130 134]
  [132 131 133]
  ...
  [208 174 161]
  [207 172 159]
  [207 172 159]]]
tf.Tensor([0], shape=(1,), dtype=int32)
